# Optimising configuration and best practices

It is rare that Splink's default settings will provide the best data linking results.  Significant improvements can usually be made with careful attention to data cleaning, comparison functions, and other configuration options.  

This notebook contains advice about best practice to help users get the more accurate data linkage results. It is based on experiences in optimising real-world jobs.

It focusses on introducing concepts and building intuition instead of providing extensive code or exhaustive details of every config option.  Please see the other examples in this repository for fully-working examples of code, and [here](https://moj-analytical-services.github.io/splink_settings_editor/) for a full list of configuration options. 

## Concepts

The [EM algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) that Splink uses to estimate model parameters is a unsupervised machine learning algorithm.  Specifically, it learns an optimal set of matching weights from the dataset of record comparisons.  

This gives us a framework for thinking about how best to use Splink because it means we can dip into the standard set tools and ways of thinking that we use for any machine learning problem.

Two things are particularly relevant for optimising of a Splink job:

- Feature engineering:  How to transform our data to help Splink learn as much as possible.
- How can we avoid confusing Splink with bad data or configuration?  (Overfitting, converging to local rather than global maxima, etc.)

A second ingredient to help structure our thoughts is an better understanding of what Splink is trying to learn.

In a nutshell, Splink looks to exploit differences in match probability between subsets of record comparisons.  The greater the differences, the more accurate the matching.

This means that the task of the user is generally to try and make these groups as different as possible (though without overfitting).

These concepts go a long way to explaining most of the optimisations in this notebook.

### Our example data

In the remainder of this notebook we will build up an example settings dictionary, based on an example dataset like this which we want to dedupe (only the first two rows are shown):


| first_name | surname | initials | gender | dob        | postcode | office           |
|------------|---------|----------|--------|------------|----------|------------------|
| robin      | linacre | rl       | M      | 2000-01-01 | TA12 9PD | Bristol          |
| john       | smith   | js       | NULL   | 1955-02-03 | BA8      | Manchester       |


### A note on default settings

Many of the settings described in this notebook have 'sensible defaults' which Splink uses if the user does not provide an explicit value.  You can always find out what these sensible defaults are by using an inbuilt function which populates all empty settings with their default values.  [This](https://github.com/moj-analytical-services/splink_demos/blob/master/code_snippets/completing_settings.py) code snippet demonstrates how to do this.
    

# Step 1. Blocking rules

The first task is to generate a dataset of pairwise record comparisons.   The main purpose of Splink is to estimate a match probability for each record comparison in this dataset.

Here is an example of a record comparison:

| first_name_l | first_name_r | surname_l | surname_r | initials_l |initials_r | gender_l | gender_r | dob_l      | dob_r      | postcode_l | postcode_r | office_l           | office_r         |
|--------------|--------------|-----------|-----------|------------|-----------|----------|----------|------------|------------|------------|------------|--------------------|------------------|
| robin        | john         | linacre   | smith     | rl         |js         | M        | NULL     | 2000-01-01 | 1955-02-03 | TA12 9PD   | BA8        | Bristol            | Manchester       |


For small datasets of perhaps around 1,000 records, it is computationally tractable to compare every record with every other.  The number of records generated will be (n * (n-1))/2, or 499,500 comparisons for 1,000 input records.  In this case, blocking rules can be ommitted entirely, and this is  probably the simplest and best way to proceed.

For larger datasets, the problem quickly becomes computationally intractable and a more efficient approach is required.  We can attempt to restrict comparisons to only records which may plausibly be matches, using an approach called 'blocking' (see [here](https://en.wikipedia.org/wiki/Record_linkage#Probabilistic_record_linkage).

For instance, if the date of birth field is always completed (no missing values) and is always accurate, then only records where date of birth matches are plausible matches.

This can be specified in the Splink settings dictionary like so:

```python

settings = {
    "link_type": "dedupe_only",
    "blocking_rules": ['l.dob = r.dob']
}
```


The eliminates all comparisons where date of birth does not match.  

In the real world, this comes with the risk that we might miss potential duplicates - either due to missingness in the date of birth field, or typos or other errors in the data.

This leads to a problem - empirically, it is usually almost impossible to find a single blocking rule which (a) reduced the number of comparisons to a managable level and (b) does not make any mistakes (i.e. does not remove any true matches).

Splink therefore allows the user to specify multiple blocking rules.

**⚠️ POTENTIAL TRAP ⚠️ :**  A lot of care must be taken when using multiple blocking rules.  The explanation for this is nuanced and relies on concepts introduced later in this notebook.  It is therefore discussed further under the header 'A deeper dive into blocking rules', below

```python

settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        'l.dob = r.dob',
        'l.office = r.office and l.initials = r.initials'
    ]
}
```

The above settings objects generates:
- all comparisons where date of birth matches
- all comparisons where both office and initials match

It then vertically concatenates these datasets, deduplicating them so that comparisons appear only once.









## Step 1. Decide on a list of comparison columns

The user must decide which columns to include in the list of `comparison_columns`.

The comparisons columns are used to subset the data into different groups - for instance, the subset of record comparisons where first name matches is likely to contain a greater proportion of matches than the subset of record comparisons where first name does not match.

It is usually best to include any columns containing information that may help us accept or reject a match.

**⚠️ POTENTIAL TRAP ⚠️ :**  Do not include columns that repeat information in other columns because this will then be double counted. In this case, the person's initials should not be included as a separate comparison column.   

Where columns are highly correlated, consider including only one, since the Fellegi Sunter model assumes independence.  Violation of this assumption can often result in some degree of double counting.  In this case, office is likely to be highly correlated with postcode, so we would advice against inclusion.


At this stage, we have the following settings

```python

settings = {
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "first_name"
        },
        {
            "col_name": "surname"
        },
        {
            "col_name": "gender"
        },
        {
            "col_name": "dob"
        },
        {
            "col_name": "postcode"
        }
    ]
}
```


## The interaction of blocking rules and comparison columns

## Step 2:  Chose the number of levels for each comparison column

Next, the user much choose the number of levels for each elements of the `comparison_columns` list.  If omitted, it defaults to 2.

Consider specifically the `first_name` and `gender` comparison columns:

```python

settings = {
    ...
    "comparison_columns": [
        {
            "col_name": "first_name",
            "num_levels": 3
        },
        ...
        {
            "col_name": "gender",
            "num_levels": 2
        },
        ...
    ]
}
```

What does `num_levels` mean and why may we want three for the name columns, but two for gender?

We are seeking to exploit differenecs in the distribution of matches between different subsets.

For a first_name field, it's reasonable to assume there may be different match rates among three groups:
- The group of record comparisons where first name matches exactly.
- The group of record comparisons where first names are similar but not exactly the same
- The group of record comparisons where first names are not similar.

`"num_levels": 3` creates these groups, enabling Splink to estimate different match weights for each group.

Another way to understand why three levels is important is to consider the cost of using only two levels:
- The group of record comparisons where first name matches exactly.
- The group of record comparisons where first name does not match exactly.

The later group contains both record comparisons where the name almost matches, and ones where names does match at all.  The algorithm has to 'take an average' - which will result in the estimated match probability for 'almost matches' being scored down too harshly, and the 'does not match at all' records not being scored down enough.

For the gender column, which contains 'M', 'F', or null, only a two-level comparison is reasonable: it either matches or it doesn't.  

### Tradeoffs

Taken to its logical extreme, we could consider having a very large number of levels to account for subtle difference in distributions between groups of records with slighly different characterstics (e.g. one group for each possible value of edit distance).

There are two downsides to increasing the number of levels:
- Overfitting.  The more parameters you have, the more likely estimates are to be influenced by specific characteristics of your training data.  With many levels, some groups will be very small, which can lead to extreme parameter estimates.
- Computational complexity.  More levels means longer compute times.

Empirically, we have found that 3 or 4 levels is generally suitable for columns where string comparison functions are being used.

## Step 3: Customising the comparison 

Until now, we have not been precise about how record comparisons are assigned to different levels.  For example, what function is used to determine whether a comparison is 'similar but not exactly the same'.

In Splink, a SQL `CASE WHEN` expression is used for this purpose.

An example of a three level statement may be:
```python
custom_case_expression = """
case
when first_name_l is null or first_name_r is null then -1
when jaro_winkler_sim(first_name_l, first_name_r) > 0.94 then 2
when jaro_winkler_sim(first_name_l, first_name_r) > 0.88 then 1
else 0 end
"""
```

This can then be provided to the settings object as follows:

```python

settings = {
    ...
    "comparison_columns": [
        {
            "col_name": "first_name",
            "num_levels": 3
            "case_expression": custom_case_expression
        },
        ...
    ]
}
```


Splink provides default case expressions when the user does not provide the setting explicitly.  **However, we recommend that the user provides a custom `case_expression` for each comparison column**

Splink provides a variety of functions that simply constructing these custom case expressions, which can be found in the `splink.case_statements` module [here](https://github.com/moj-analytical-services/splink/blob/master/splink/case_statements.py).

For example, to generate the above statement, the user could have done this:

```python
from splink.case_statements import sql_gen_gammas_case_stmt_jaro_3
print(sql_gen_gammas_case_stmt_jaro_3('first_name'))
```

#### Writing an effective case expression

Recall that we want to create subsets of record comparisons which have different proportions of matches vs non matches - the more different the better.

For example, consider the case of a three-level case expression for first name.

We know that names can often be misspelt, or nicknames or diminutive forms may be used (e.g. Robin vs Rob).  

It therefore probably does not make sense to group comparisons of Robin vs Rob with comparisons where there is no similarity (Robin vs John) because the former group is likely to have a reasonably high match rate, whereas the later group is likely to have a low match rate. 

We therefore want to craft a case statement that 'picks out' the relevant comparisons as precisely as possible.  

For instance, we probably want to distinguish between 'Robin vs Rob' and 'Susie' vs 'Julie'.

By choosing the right string comparison function we can ensure that 'Robin vs Rob' is assigned to level 1 (some similarity), and 'Susie' vs 'Julie' is assigned to level 0 (no similarity).

### String comparison functions provided with Splink

A variety of common string comparison functions are provided in a [jar](https://github.com/moj-analytical-services/splink/tree/master/jars) accompanying splink. Since these are implmeneted in Java/Scala, they should be much faster than using equivalent Python functions.

A code snippet showing how to load them into Spark can be found [here](https://github.com/moj-analytical-services/splink_demos/blob/master/code_snippets/loading_jar.py). 

The functions provided are:
- **[Jaro-Winkler similarity](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance#Jaro%E2%80%93Winkler_Similarity).**  A normalised edit distance running from 0 (no match) to 1 (exact match). Places greater weight on earlier characters. Generally a good option for names and dictionary words.
- [**Jaccard similarity**](https://en.wikipedia.org/wiki/Jaccard_index).  A token-based similarity measure that considers the number of common tokens.  May be appropriate for some unique identifiers such as a driving licence number that may be entered with error.
- [**Cosine distance**](https://en.wikipedia.org/wiki/Cosine_similarity).  A measure of similarity between different substrings.  Particularly appropriate for longer text strings that cannot be parsed out into separate fields.

Splink defaults to using [Jaro-Winkler similarity](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance#Jaro%E2%80%93Winkler_Similarity) for string comparisons, which is generally a good choice for names or dictionary words.







### Custom comparisons using more than one column

We previously noted that the list of comparison columns should not contain multiple columns that measure the same thing, because this causes double counting in the match probability estimate.    

For instance, in our example dataset the `office` and `postcode` fields are both essentially measures of the location of a person at different levels of granularity, so they should not be included as separate comparison columns.

However, it's not uncommon to want to include information from both columns because a _combiantion_ of the columns provides a more accurate measure of location than either individual column.  In this example, perhaps the postcode column is often empty, but the office field is always complete.  We want to use postcode as a measure of location where the data exists, and fall back on office when the postcode is missing.

We achieve this in Splink by passing a list of `custom_columns_used`, associated with a `custom_name` which is chosen by the user.  



```python
custom_case_expression = f"""
case
when (office_l is null or office_r is null) and (postcode_l is null or postcode_r is null) then -1
when (postcode_l is not null and postcode_r is not null) and postcode_l = postcode_r then 2
when office_l = office_r then 1
else 0 end

"""

settings = {...
            "comparison_columns": [
                {            
                "custom_name": "location_custom_comparison"
                "case_expression": custom_case_expression,
                "custom_columns_used": ["postcode", "office"],
                "num_levels": 3
                }
                ]
           }
``` 

## Identifying problems

Convergence issues

Odd graphical output

## A deeper dive into multiple blocking rules

Good blocking rules eliminate large numbers of non-matching record comparisons without eliminating record comparisons which do match.  That is, we are looking for a decision rule with 100% recall, and with good enough sensitivity to reduce record comparisons to computationally tractable number.

Unfortunately, it is rarely possible to find blocking rules which meet both of these criteria.  For example, blocking on date of birth will greatly reduce the number of record comparisons, but if typos are present in the date of birth field, the rule may eliminate a small number of true matches.

This problem can be remedied by considering multiple blocking rules.  For instance, we may block on date of birth OR postcode.  Now the only true matches we miss are matches where there is an error in both the postcode and the date of birth.

There are two broad approaches possible here:
1. Running separate data matching exercise for each blocking rule.
2. Generating a list of record comparisons which match any of the blocking rules, and running a single Splink job to estimate matching weights across all these record comparisons.

Option 1 is theoretically more sound and is likely to result in more accurate matching results.  However, it results in significantly more complexity, because different settings are needed for each job, diagnostics need to be run on each job separately to check parameter estimates are sensible,  and the results from multiple jobs then need to be combined.
Option 2 is much simpler and can often yield results which are 'good enough'.  However, in some situations it can result in completely nonsensical results.

To understand why option 2 can be a bad idea, consider running a single Splink job whether we block on:

Rule 1: Social security number 
Rule 2: Date of birth.  

Assume that both fields are a result of manual data entry and are subject to transciption errors.

The problem is that _same_ parameter estimates have to apply to all record comparisons, irrespective of the blocking rule that generated them.  

This means we are likely to underestimate match probability for records 

To understand this, consider the subset of record comparisons in which date of birth does not match.  These record comparison must arise from blocking rule 1, which in turn means that social security number matches for all records in this subset.  As such, they are highly likely to be matches.  A non-match on date of birth is likely to be evidence _in favour_ of a match.  This implies that a match on date of birth is evidence _against_ a match.  

This then leads to a result which is clearly incorrect. Compare the following two records comparisons:
Comparison 1:  Social security number and date of birth match
Comparison 2:  Social security number match, date of birth does not match

Intuitively we expect comparison 1 to get a higher match probability, but in fact, comparison 2 is scored higher.

This proves there are cases in which multiple blocking rules is a bad idea.  However, it does not prove it is always a bad idea - and we have found that, empirically, there are cases where multiple blocking rules are effective.


### Empirical advice: When to use multiple blocking rules


It's difficult to give advice on multiple blocking which applies to all situations.  The following rules of thumb may be useful:

- Multiple blocking rules can lead to bad results when different blocking rules imply big differeces in conditional probabilities.  For example, a match on 'surname' may be much more informative when blocking on date of birth than when blocking on post code.
- Multiple blocking rules seem to work better when the proportion of matches and non matches generated by each blocking rule is similar
- The problems associated with multiple blocking rules seem more acute the fewer the number of comparison columns you have.  

Because of the problems associated with multiple blocking rules, very careful attention should be taken to convergence and parameter estimates. 

One particuar situation in which have found the use of multiple blocking rules is effective is when strugging to find a single blocking rule that is appropriately strict.

For instance, suppose blocking on postcode alone, or date of birth alone, result in too many record comparisons.

In this case we may consider a series of blocking rules such as:

```python

settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        'l.dob = r.dob and l.forename_initial = r.forename_initial',
        'l.dob = r.dob and l.surname_initial = r.surname_initial',
        'l.dob = r.dob and l.outer_postcode = r.outer_postcode'
    ]
}
```

Which restricts comparisons down to a more managable number, whilst being very unlikely to reject true matches amongst the dte of birth block.



```python

settings = {
    ...
    "comparison_columns": [
        {
            "col_name": "first_name",
            "num_levels": 3,
            "u_probabilities": [
                    0.7,
                    0.2,
                    0.1
                ],
            "m_probabilities": [
                    0.1,
                    0.2,
                    0.7
                ]
        },
        ...
    ]
}



In [ ]:
# Other considerations

In [9]:
## Term frequency adjustments 